In [1]:
import numpy as np

In [1]:
import sys, tensorflow as tf, platform, os
print("Python:", sys.version)
print("Executable:", sys.executable)
print("TF:", tf.__version__)
print("VIRTUAL_ENV:", os.environ.get("VIRTUAL_ENV"))


d:\progs\Python\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\progs\Python\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\progs\Python\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  

Python: 3.13.6 (tags/v3.13.6:4e66535, Aug  6 2025, 14:36:00) [MSC v.1944 64 bit (AMD64)]
Executable: d:\progs\Python\python.exe
TF: 2.20.0-rc0
VIRTUAL_ENV: None


In [2]:
import sys, subprocess, pathlib
req = pathlib.Path("requirements_exact.txt")
subprocess.check_call([sys.executable, "-m", "pip", "freeze"], stdout=req.open("w"))
print("נוצר requirements_exact.txt")



נוצר requirements_exact.txt


In [2]:
data=np.load('../data/NTU60_CS.npz')
x_train=data['x_train']
x_test=data['x_test']
y_train=data['y_train']
y_test=data['y_test']

In [3]:
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')


In [4]:
max_val = np.max(np.abs(x_train))
x_train /= max_val
x_test /= max_val

In [5]:
x_train = x_train.astype('float16')
x_test = x_test.astype('float16')


In [6]:

print("Max value used for normalization:", max_val)
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)


Max value used for normalization: 5.289812
x_train shape: (40091, 300, 150) y_train shape: (40091, 60)


In [7]:
import tensorflow as tf 
from tensorflow.keras import layers, models

d:\progs\Python\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\progs\Python\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\progs\Python\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/resource_handle.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  

In [8]:
model = models.Sequential(
    [
        layers.Masking(mask_value=0.0, input_shape=(300, 150)),
        layers.LSTM(128, return_sequences=True),
        layers.LSTM(128),
        layers.Dense(60, activation='softmax'),
    ]
)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

d:\progs\Python\Lib\site-packages\keras\src\layers\core\masking.py:48: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 300, 150)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 300, 128)       │       142,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 60)             │         7,740 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 282,172 (1.08 MB)

 Trainable params: 282,172 (1.08 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
batch_size = 16
shuffle_buf = 4096
epochs = 10

# יצירת Dataset לאימון עם ערבוב ואצוות קטנות
train_ds =tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(buffer_size=shuffle_buf) \
                   .batch(batch_size) \
                   .prefetch(tf.data.AUTOTUNE)




In [10]:
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [ ]:
history = model.fit(train_ds, epochs=epochs, validation_data=test_ds)

Epoch 1/10
2506/2506 ━━━━━━━━━━━━━━━━━━━━ 3322s 1s/step - accuracy: 0.2602 - loss: 2.5768 - val_accuracy: 0.3776 - val_loss: 2.0977
Epoch 2/10
1230/2506 ━━━━━━━━━━━━━━━━━━━━ 29:30 1s/step - accuracy: 0.4055 - loss: 1.9917